ขั้นตอนนี้คือโหลด dataset จาก .pt เอามาดู x, y shape

In [7]:
import torch

MODEL_PATH = "/Users/sarawit/Documents/Year2/sem2/Artificial-inteligent/FINAL_PROJECT/fnpj/csi_dataset/csi_windows_w64_s32.pt"

data = torch.load(MODEL_PATH, map_location="cpu")

X_all = data["X"]   # expected shape: (N, 64, F)
y_all = data["y"]   # expected shape: (N,)

print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)

X_all shape: torch.Size([173, 32, 53])
y_all shape: torch.Size([173])


In [8]:
import numpy as np
unique, counts = np.unique(y_all, return_counts=True)
print("classes:", unique)
print("counts :", counts)

classes: [0 1 2]
counts : [21 79 73]


This code performs cross-validation-like evaluation by training and testing an LSTM model multiple times with different random seeds to assess performance stability. It iterates over a list of seeds, splits the data into train/test sets using stratified sampling, creates new datasets and data loaders each time, initializes a fresh model, trains it for 10 epochs, and then evaluates accuracy on the test set. This helps check if the model's performance varies significantly with different data splits.

In [9]:
from sklearn.model_selection import train_test_split
import numpy as np

seeds = [0, 1, 2, 3, 4, 5, 10, 42, 99]

for seed in seeds:
    print("\n==============================")
    print("Random seed:", seed)

    # 1️⃣ split ใหม่ทุกครั้ง
    X_train, X_test, y_train, y_test = train_test_split(
        X_all, y_all,
        test_size=0.2,
        random_state=seed,
        stratify=y_all
    )

    # 2️⃣ สร้าง dataset + dataloader ใหม่
    train_ds = CSIDataset(X_train, y_train)
    test_ds  = CSIDataset(X_test, y_test)

    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False)

    # 3️⃣ สร้าง model ใหม่ทุกครั้ง (สำคัญ!)
    model = LSTMClassifier(input_size=53, hidden_size=128, num_layers=2, num_classes=3)
    model = model.to(DEVICE)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # 4️⃣ train แค่ 10 epoch พอ
    for epoch in range(10):
        model.train()
        for xb, yb in train_loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

    # 5️⃣ evaluate test
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for xb, yb in test_loader:
            xb = xb.to(DEVICE)
            logits = model(xb)
            preds = logits.argmax(dim=1).cpu()
            correct += (preds == yb).sum().item()
            total += yb.size(0)

    acc = correct / total
    print("Test accuracy:", acc)


Random seed: 0


NameError: name 'CSIDataset' is not defined

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

# เลือก device (ถ้ามี MPS บน Mac ก็ใช้)
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
print("Device:", DEVICE)

class CSIDataset(Dataset):
    def __init__(self, X, y):
        # X, y ตอนนี้เป็น torch.Tensor อยู่แล้วจาก train_test_split
        # ถ้าไม่ใช่ tensor ให้ครอบด้วย torch.tensor(...)
        self.X = X.float()
        self.y = y.long()

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

BATCH_SIZE = 32

train_ds = CSIDataset(X_train, y_train)
test_ds  = CSIDataset(X_test,  y_test)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)

print("Train batches:", len(train_loader))
print("Test  batches:", len(test_loader))

In [ ]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, num_classes=3):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True   # ทำให้ shape เป็น (batch, seq, feature)
        )
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # x: (B, T, F) = (batch_size, sequence_length, num_features)
        out, _ = self.lstm(x)        # out: (B, T, H)
        last = out[:, -1, :]         # เอา timestep สุดท้าย: (B, H)
        logits = self.fc(last)       # (B, num_classes)
        return logits

input_size = X_all.shape[2]  # = 53
model = LSTMClassifier(input_size=input_size, hidden_size=128, num_layers=2, num_classes=3)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()            # baseline ยังไม่ใส่ class weight
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

print(model)

In [ ]:
for epoch in range(1, EPOCHS + 1):

    model.train()
    total_loss = 0.0
    correct = 0
    total = 0

    # ===== TRAIN =====
    for xb, yb in train_loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)

        optimizer.zero_grad()
        logits = model(xb)
        loss = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * yb.size(0)
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    epoch_acc = correct / total


    print(f"Epoch {epoch:02d} | train_acc={epoch_acc:.4f}")

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt

model.eval()
all_preds = []
all_true = []

with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(DEVICE)
        logits = model(xb)
        preds = logits.argmax(dim=1).cpu().numpy()

        all_preds.extend(preds)
        all_true.extend(yb.numpy())

all_preds = np.array(all_preds)
all_true = np.array(all_true)

cm = confusion_matrix(all_true, all_preds, labels=[0,1,2])
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n")
print(classification_report(all_true, all_preds, digits=3, target_names=["no_human","static","movement"]))

# plot confusion matrix ให้ดูเหมือนของเพื่อน
plt.figure(figsize=(5,4))
plt.imshow(cm, cmap="viridis")
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, str(cm[i, j]), ha="center", va="center", color="white")

plt.xticks([0,1,2], [0,1,2])
plt.yticks([0,1,2], [0,1,2])
plt.tight_layout()
plt.show()